# 콘텍스트 관리자와 else 블록

* with문과 콘텍스트 관리자
* for, while, try 문에서 else 블록

## 15.1 이것 다음에 저것: if문 이외에서의 else블록


In [ ]:
try:
    dangerous_call()
except OSError:
    log('OSError...')
else:
    after_call()

## 15.2 콘텍스트 관리자와 with 블록
* with문은 try/finally 패턴을 단순화하기 위해 설계되었다.

In [ ]:
# 콘텍스트 관리자로서 파일 객체의 사용 예
with open('mirror.py') as fp:
    src = fp.read(60)
    
len(src)
fp # fp 변수는 여전히 살아있다.
fp.closed, fp.encoding # fp객체의 속성을 읽을 수 있다.
fp.read(60) # 그러나 with블록이 끝날 때 TextIOWrapper.__exit__()메서드가 호출되어 파일을 닫으므로, fp를 이용해서 파일 입출력을 할 수 없다.

In [ ]:
#LookingGlass 콘텍스트 관리자 클래스의 시험 주행
from mirror import LookingGlass
with LookingGlass() as what: # 파이썬은 콘텍스트 관리자의 __enter__() 메서드를 호출해서 반환
        print('Alice, Kitty and Snowdrop') # 문자열을 출력하고 나서 what의 값을 출력한다.
        print(what)
# print()가 문자열을 역순으로 출력한다.
what #이제 with 블록이 끝났으니, __enter__()가 반환해서 what에 저장한 문자열 'JABBERWOCKY'를 제대로 출력할 수 있다.
print('Back to normal') #이제 print()가 정상적으로 출력한다.

In [ ]:
class LookingGlass:
    
    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY'
    
    def reverse_write(self, text):
        self.original_write(text[::-1])
        
    def __exit__(self, exc_type, exc_value, traceback):
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True

1. exc_type: ZeroDivisionError 등의 예외 클래스
2. exc_value: 예외 객체. 예외 메세지 등 exception() 생성자에 전달된 인수는 ecx_value.args 속성을 이용해서 볼 수 있다.
3. traceback: traceback 객체

In [ ]:
from mirror import LookingGlass
manager = LookingGlass() # manager 객체를 생성하고 조사한다.
manager
monster = manager.__enter__() # 콘텍스트 관리자의 __enter__() 메서드를 호출하고 결과를 monster에 저장한다.
manager.__exit__(None, None, None) # manager.__exit__() 메서드를 호출해서 stdout.write() 메서드를 복원한다.

## 15.3 contextlib 유틸리티

* closing(): close 메서드는 제공하지만 _enter_()/_exit_() 프로토콜을 구현하지 않는 객체로부터 콘텍스트 관리자를 생성하는 함수

* suppress: 지정한 예외를 임시로 무시하는 콘텍스트 관리자

* @contextmanager: 클래스를 생성하고 프로토콜을 구현하는 대신, 간단한 제너레이터 함수로부터 콘텍스트 관리자를 생성할 수 있게 해주는 데커레이터

* ContextDecorator: 콘텍스트 관리자를 함수 데커레이터로도 사용할 수 있게 해주는 기반 클래스

* ExitStack: 여러 콘텍스트 관리자를 입력할 수 있게 해주는 콘텍스트 관리자. with 블록이 끝나면 ExitStack은 누적된 콘텍스트 관리자들의 _exit_() 메서드를 LIFO 순서(마지막에 들어간 것이 처음 나온다)로 호출한다. 예를 들어 임의의 파일 리스트에 있는 파일을 한꺼번에 여는 경우처럼, with 블록 안에 들어가기 전에 얼마나 많은 콘텍스트 관리자가 필요한지 사전에 알 수 없을 때 이 클래스를 사용.

## 15.4 @contextmanager 사용하기
- @contextmanager 데커레이터는 콘텍스트 관리자를 생성할 때 작성하는 틀에 박힌 코드를 줄여준다. _enter_()와 _exit_() 메서드를 가진 클래스 전체를 작성하는 대신 _enter_() 메서드가 반환할 것을 생성하는 yield문 하나를 가진 제너레이터만 구현하면 된다.

In [ ]:
#mirror_gen.py:제너레이터로 구현한 콘텍스트 관리자
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write
    yield 'JABBERWOCKY' #with 문의 as 절에 있는 타깃 변수에 바인딩될 값을 생성. with문의 본체가 실행되는 동안 이 함수는 여기에서 실행을 일시 중단한다.
    sys.stdout.write = original_write # 제어 흐름이 with 블록을 빠져나오면 yield문 이후의 코드가 실행된다. 여기서는 원래의 sys.stdout.write() 메서드를 복원한다.

In [ ]:
from mirror_gen import looking_glass
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)